In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib notebook

In [3]:
st_iter = pd.read_csv("dataset/salida2.csv", chunksize=1000000)

## Cantidad de bicis

Queremos saber cuántas bicicletas hay en función del tiempo. Para eso tenemos:

- en status.csv disponemos de la cantidad de bicicletas disponibles en un minuto dado.
- en trips.csv tenemos los viajes en todo momento.

Podemos medir, a las 3 am de todos los días (para minimizar ruido) cuántas bicicletas hay. **NOTA** Podemos usar un _daterange para esto_.

### Análisis previo: a qué horas hay más bicis en viaje?

Para medir esto levantamos trips.csv y analizamos cada media hora cuántos viajes hay. Primero hagámoslo a las 3. Después vemos cómo lo mejoramos para que sea en todos.

In [6]:
viajes = pd.read_csv("dataset/trip.csv")

In [7]:
# Veamos qué estilo tiene.
viajes.head(3)

,id,duration,start_date,start_station_name,start_station_id,end_date,end_station_name,end_station_id,bike_id,subscription_type,zip_code
0,4576,63,8/29/2013 14:13,South Van Ness at Market,66,8/29/2013 14:14,South Van Ness at Market,66,520,Subscriber,94127
1,4607,70,8/29/2013 14:42,San Jose City Hall,10,8/29/2013 14:43,San Jose City Hall,10,661,Subscriber,95138
2,4130,71,8/29/2013 10:16,Mountain View City Hall,27,8/29/2013 10:17,Mountain View City Hall,27,48,Subscriber,97214


In [8]:
# Y qué tipos.
viajes.dtypes

id                     int64
duration               int64
start_date            object
start_station_name    object
start_station_id       int64
end_date              object
end_station_name      object
end_station_id         int64
bike_id                int64
subscription_type     object
zip_code              object
dtype: object

### Arreglemos las fechas.
Esto debería ser cuestión de, al leer, especificar fechas. Por ahí sea importante tener en cuenta que el formato de fecha es mes/dia/año (oh god why).

Se le especifica que infiera los formatos de fecha, lo cual acelera considerablemente el proceso.

In [2]:
viajes = pd.read_csv("dataset/trip.csv", parse_dates=["start_date", "end_date"], 
                     infer_datetime_format=True)
viajes.dtypes

id                             int64
duration                       int64
start_date            datetime64[ns]
start_station_name            object
start_station_id               int64
end_date              datetime64[ns]
end_station_name              object
end_station_id                 int64
bike_id                        int64
subscription_type             object
zip_code                      object
dtype: object

In [63]:
viajes.end_date.head(3)

0   2013-08-29 14:14:00
1   2013-08-29 14:43:00
2   2013-08-29 10:17:00
Name: end_date, dtype: datetime64[ns]

Perfecto, las fechas fueron parseadas y comprobamos que se corrigieron al formato japonés.

### Cosas que suceden a las 4 de la mañana
¿Cómo se filtra una columna por fechas? Nuestra idea es que queden en la columna las cosas que arrancan antes de las 4:00 y terminan después de las 4:00. Tenemos que comparar fechas.

In [37]:
a = np.datetime64("2013-01-01 09:59:59")
b = np.datetime64("2013-01-01 10:00:00")
c = np.datetime64("2013-01-01 10:00:01")

a < b < c

True

Tres fechas cualesquiera se pueden comparar desde arriba. Y si solo nos importa el **tiempo/hora** y no **el día** para filtrar?

In [58]:
a = np.datetime64("2013-01-01 10:00:00")
b = np.datetime64("2013-01-02 10:00:00")

print(a<b)
print(a.tolist().time() < b.tolist().time())

True
False


Perfecto. Esto nos sirve para ver si algo está a las 4 de la mañana. Intentamos filtrar por esa condición.

In [92]:
viajes4am = viajes[(viajes.start_date.dt.hour < 4) & (4 <= (viajes.end_date.dt.hour))]
len(viajes4am)

289

In [99]:
viajes4am = viajes4am.reset_index()

**Problema**: Esas no son todas. Una bicicleta puede haber sido alquilada por 3 días. Podemos hacer algo al estilo map-reduce.

1. Map: Por cada una se hace una lista con las horas por las que pasa.
2. ReduceByKey: Se cuenta la cantidad de apariciones de cada hora.

In [257]:
# Suma de apariciones de cada hora en las listas.
suma = [0]*24

def horas(viaje):
    # Obtengo la hora más cercana por arriba al inicio.
    startf = viaje.start_date.ceil("H")
    # Hora más cercana por abajo al fin.
    endf = viaje.end_date.floor("H")
    rango = pd.date_range(start=startf, end=endf, freq="H").hour
    # Sumamos 1 a las apariciones de cada hora en el rango.
    for h in rango:
        suma[h]+=1

Dato interesante: cuando un apply devuelve una serie, se crea una columna por cada elemento, permitiendo labels. Esto puede ser muy util para otros casos. Acá es peligroso porque se genera un dataframe disperso con muchísimas columnas.

In [258]:
viajes.apply(horas, axis=1);

KeyboardInterrupt: 

In [255]:
suma

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [ ]:
viajes.head(5).apply


### Zip_codes
_A ver en otro momento si es necesario_
Por que el zip code no figura como numérico? vamos a ver casos que no lo sean.

In [19]:
zip_codes_fallados = viajes.loc[pd.to_numeric(viajes.zip_code, errors="coerce").isnull(), "zip_code"]

In [21]:
zip_codes_fallados.value_counts()

nil           10682
94040-1724      275
94103-2585       60
v6z2x            25
94107-3471       10
946-2             4
M4S1P             2
Name: zip_code, dtype: int64

Bueno, por ahora no vamos a tocar los zip-codes. No nos sirven. Y son alto bardo. Algunos no están implícitamente, otros no están explícitamente (nil, son muchos) y otros son compuestos.